In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/tmp/ipykernel_1210518/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
model_path = "gpt2-100M/checkpoint-100M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_100M_whitespace"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    #print(token_surprisals)

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 4.9647111892700195),
 ('that', 3.957824468612671),
 ('your', 7.061441898345947),
 ('friend', 6.687188148498535),
 ('gave', 8.715492248535156),
 ('a', 5.011169910430908),
 ('baguette', 42.164002418518066),
 ('to', 4.279263973236084),
 ('Mary', 13.89819049835205),
 ('last', 8.088367462158203),
 ('weekend', 7.8527045249938965),
 ('.', 1.655366063117981)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(relevant_surprisals)

In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df

def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    '''    
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')
    '''

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [8]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [9]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['gap_distance'] = pd.Categorical(sentence_df['gap_distance'], categories = ['null', 'short', 'medium', 'long'], ordered = True)

Gap Position == Object

In [10]:
# modifier: short_mod, med_mod, long_mod

In [11]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [12]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 17.7311
Min. group size: 16
Log-Likelihood: -930.1626
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.042,1.043,21.141,0.000,19.999,24.086
1,gap_distance[T.short],-0.180,0.666,-0.270,0.787,-1.485,1.125
2,gap_distance[T.medium],-1.675,0.666,-2.516,0.012,-2.980,-0.370
3,gap_distance[T.long],-1.305,0.666,-1.960,0.050,-2.610,0.000
4,wh_numeric,0.291,0.471,0.619,0.536,-0.632,1.214
5,wh_numeric:gap_distance[T.short],-0.126,0.666,-0.190,0.850,-1.431,1.179
6,wh_numeric:gap_distance[T.medium],-0.227,0.666,-0.342,0.733,-1.532,1.077
7,wh_numeric:gap_distance[T.long],-0.244,0.666,-0.367,0.713,-1.549,1.060
8,gap_numeric,-1.969,0.471,-4.183,0.000,-2.892,-1.047
9,gap_numeric:gap_distance[T.short],0.071,0.666,0.107,0.915,-1.234,1.376



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,17.309,1.466


In [13]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 17.3350
Min. group size: 16
Log-Likelihood: -951.1251
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.010,1.002,21.962,0.000,20.045,23.974
1,wh_numeric,0.257,0.370,0.694,0.488,-0.469,0.982
2,gap_numeric,-2.339,0.370,-6.319,0.000,-3.065,-1.614
3,wh_numeric:gap_numeric,-1.264,0.370,-3.414,0.001,-1.989,-0.538
4,gap_distance_length,-0.021,0.008,-2.611,0.009,-0.037,-0.005
5,wh_numeric:gap_distance_length,-0.003,0.008,-0.400,0.689,-0.019,0.013
6,gap_numeric:gap_distance_length,0.014,0.008,1.715,0.086,-0.002,0.030
7,wh_numeric:gap_numeric:gap_distance_length,0.015,0.008,1.892,0.059,-0.001,0.031



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,17.321,1.480


Gap Postion == PP/Goal

In [14]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [15]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 30.0501
Min. group size: 16
Log-Likelihood: -1005.6765
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.457,1.112,18.400,0.000,18.278,22.636
1,gap_distance[T.short],-0.400,0.867,-0.462,0.644,-2.099,1.298
2,gap_distance[T.medium],-0.419,0.867,-0.483,0.629,-2.118,1.280
3,gap_distance[T.long],-0.625,0.867,-0.721,0.471,-2.324,1.074
4,wh_numeric,-0.066,0.613,-0.108,0.914,-1.268,1.135
5,wh_numeric:gap_distance[T.short],0.001,0.867,0.001,0.999,-1.698,1.700
6,wh_numeric:gap_distance[T.medium],0.025,0.867,0.029,0.977,-1.674,1.724
7,wh_numeric:gap_distance[T.long],0.057,0.867,0.066,0.947,-1.641,1.756
8,gap_numeric,5.606,0.613,9.147,0.000,4.405,6.807
9,gap_numeric:gap_distance[T.short],-0.730,0.867,-0.842,0.400,-2.429,0.969



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,17.208,1.167


In [16]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 29.3873
Min. group size: 16
Log-Likelihood: -1028.8353
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.308,1.049,19.358,0.000,18.252,22.364
1,wh_numeric,-0.065,0.482,-0.135,0.893,-1.010,0.880
2,gap_numeric,5.395,0.482,11.193,0.000,4.450,6.339
3,wh_numeric:gap_numeric,-0.294,0.482,-0.609,0.542,-1.238,0.651
4,gap_distance_length,-0.006,0.011,-0.563,0.573,-0.027,0.015
5,wh_numeric:gap_distance_length,0.001,0.011,0.052,0.958,-0.020,0.021
6,gap_numeric:gap_distance_length,0.003,0.011,0.254,0.799,-0.018,0.023
7,wh_numeric:gap_numeric:gap_distance_length,0.003,0.011,0.301,0.764,-0.018,0.024



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,17.323,1.184


Embedded Clause

In [17]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [18]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [19]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2329
Min. group size: 16
Log-Likelihood: -275.1315
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.285,0.140,51.887,0.000,7.010,7.561
1,gap_distance[T.short],0.767,0.076,10.058,0.000,0.618,0.917
2,gap_distance[T.medium],0.390,0.076,5.118,0.000,0.241,0.540
3,gap_distance[T.long],1.008,0.076,13.217,0.000,0.859,1.158
4,wh_numeric,0.014,0.054,0.269,0.788,-0.091,0.120
5,wh_numeric:gap_distance[T.short],0.137,0.076,1.796,0.072,-0.012,0.287
6,wh_numeric:gap_distance[T.medium],0.119,0.076,1.557,0.119,-0.031,0.268
7,wh_numeric:gap_distance[T.long],0.086,0.076,1.127,0.260,-0.064,0.236
8,gap_numeric,-0.084,0.054,-1.550,0.121,-0.189,0.022
9,gap_numeric:gap_distance[T.short],0.085,0.076,1.119,0.263,-0.064,0.235



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.336,0.243


In [20]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2935
Min. group size: 16
Log-Likelihood: -315.7996
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.456,0.137,54.576,0.000,7.188,7.724
1,wh_numeric,0.072,0.048,1.491,0.136,-0.023,0.166
2,gap_numeric,-0.069,0.048,-1.426,0.154,-0.163,0.026
3,wh_numeric:gap_numeric,-0.127,0.048,-2.637,0.008,-0.221,-0.033
4,gap_distance_length,0.010,0.001,9.834,0.000,0.008,0.013
5,wh_numeric:gap_distance_length,0.001,0.001,0.752,0.452,-0.001,0.003
6,gap_numeric:gap_distance_length,0.002,0.001,1.622,0.105,-0.000,0.004
7,wh_numeric:gap_numeric:gap_distance_length,0.002,0.001,1.741,0.082,-0.000,0.004



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.326,0.213


Embedded Clause - PP/Goal Gap Position

In [21]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [22]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2315
Min. group size: 16
Log-Likelihood: -273.4068
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.568,0.135,56.133,0.000,7.304,7.832
1,gap_distance[T.short],0.597,0.076,7.851,0.000,0.448,0.746
2,gap_distance[T.medium],0.241,0.076,3.172,0.002,0.092,0.390
3,gap_distance[T.long],0.808,0.076,10.617,0.000,0.659,0.957
4,wh_numeric,0.039,0.054,0.722,0.470,-0.067,0.144
5,wh_numeric:gap_distance[T.short],0.014,0.076,0.186,0.852,-0.135,0.163
6,wh_numeric:gap_distance[T.medium],0.032,0.076,0.420,0.675,-0.117,0.181
7,wh_numeric:gap_distance[T.long],0.008,0.076,0.103,0.918,-0.141,0.157
8,gap_numeric,0.317,0.054,5.901,0.000,0.212,0.423
9,gap_numeric:gap_distance[T.short],-0.065,0.076,-0.858,0.391,-0.214,0.084



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.306,0.223


In [23]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2703
Min. group size: 16
Log-Likelihood: -303.0333
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.684,0.132,58.399,0.000,7.426,7.942
1,wh_numeric,0.048,0.046,1.035,0.300,-0.043,0.138
2,gap_numeric,0.302,0.046,6.542,0.000,0.212,0.393
3,wh_numeric:gap_numeric,-0.018,0.046,-0.388,0.698,-0.109,0.073
4,gap_distance_length,0.008,0.001,8.165,0.000,0.006,0.010
5,wh_numeric:gap_distance_length,0.000,0.001,0.124,0.902,-0.002,0.002
6,gap_numeric:gap_distance_length,-0.002,0.001,-1.559,0.119,-0.004,0.000
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.001,0.281,0.779,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.303,0.206


#### Double Gaps

In [24]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [25]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']

In [26]:
sentence_df['gap_count'] = pd.Categorical(sentence_df['gap_count'], categories = [1, 0, 2], ordered = True)

Embedded Clause

In [27]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [28]:
iv_list = ['wh_numeric', 'gap_count']
result = mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed')
result


=== DOUBLE GAP EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3010
Min. group size: 8
Log-Likelihood: -169.8905
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.357,0.238,30.860,0.000,6.890,7.824
1,gap_count[T.0],-0.582,0.106,-5.481,0.000,-0.791,-0.374
2,gap_count[T.2],1.289,0.106,12.131,0.000,1.081,1.497
3,wh_numeric,-0.077,0.061,-1.251,0.211,-0.197,0.044
4,wh_numeric:gap_count[T.0],0.322,0.106,3.027,0.002,0.113,0.530
5,wh_numeric:gap_count[T.2],-0.001,0.106,-0.009,0.993,-0.209,0.207



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.061,0.694


#### Wh-Islands

In [29]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [30]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)

Post-Gap Region - Continuation

In [31]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [32]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation')


=== WH-ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 32.4209
Min. group size: 12
Log-Likelihood: -764.3407
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,19.541,1.079,18.115,0.000,17.427,21.655
1,comp_type[T.that],0.041,0.900,0.045,0.964,-1.724,1.805
2,comp_type[T.whether],0.053,0.900,0.059,0.953,-1.711,1.818
3,wh_numeric,-0.043,0.637,-0.067,0.946,-1.291,1.205
4,wh_numeric:comp_type[T.that],-0.242,0.900,-0.269,0.788,-2.007,1.522
5,wh_numeric:comp_type[T.whether],-0.001,0.900,-0.001,0.999,-1.766,1.763
6,gap_numeric,3.901,0.637,6.128,0.000,2.653,5.149
7,gap_numeric:comp_type[T.that],0.781,0.900,0.867,0.386,-0.984,2.545
8,gap_numeric:comp_type[T.whether],1.509,0.900,1.676,0.094,-0.255,3.274
9,wh_numeric:gap_numeric,-0.971,0.637,-1.526,0.127,-2.219,0.276



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,15.167,1.063


Embedded Clause

In [33]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [34]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed')


=== WH-ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0632
Min. group size: 12
Log-Likelihood: -85.5908
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.501,0.233,36.488,0.000,8.044,8.957
1,comp_type[T.that],-0.743,0.040,-18.694,0.000,-0.821,-0.665
2,comp_type[T.whether],0.066,0.040,1.659,0.097,-0.012,0.144
3,wh_numeric,-0.016,0.028,-0.569,0.569,-0.071,0.039
4,wh_numeric:comp_type[T.that],0.017,0.040,0.424,0.671,-0.061,0.095
5,wh_numeric:comp_type[T.whether],-0.062,0.040,-1.562,0.118,-0.140,0.016
6,gap_numeric,0.280,0.028,9.947,0.000,0.225,0.335
7,gap_numeric:comp_type[T.that],-0.015,0.040,-0.385,0.700,-0.093,0.063
8,gap_numeric:comp_type[T.whether],0.114,0.040,2.870,0.004,0.036,0.192
9,wh_numeric:gap_numeric,-0.103,0.028,-3.658,0.000,-0.158,-0.048



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.070,1.449


#### Adjunct Islands 

In [35]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [36]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [37]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)

Post-Gap Region - Continuation

In [38]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [39]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation')


=== ADJUNCT ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 27.8071
Min. group size: 12
Log-Likelihood: -752.1802
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,27.752,1.255,22.106,0.000,25.291,30.212
1,adjunct_type[T.back],-1.137,0.834,-1.364,0.173,-2.772,0.497
2,adjunct_type[T.front],-0.421,0.834,-0.504,0.614,-2.055,1.214
3,wh_numeric,0.409,0.590,0.694,0.488,-0.746,1.565
4,wh_numeric:adjunct_type[T.back],-0.348,0.834,-0.418,0.676,-1.983,1.286
5,wh_numeric:adjunct_type[T.front],-0.350,0.834,-0.419,0.675,-1.984,1.285
6,gap_numeric,4.365,0.590,7.403,0.000,3.209,5.520
7,gap_numeric:adjunct_type[T.back],1.229,0.834,1.475,0.140,-0.405,2.864
8,gap_numeric:adjunct_type[T.front],0.843,0.834,1.012,0.312,-0.791,2.478
9,wh_numeric:gap_numeric,-0.819,0.590,-1.390,0.165,-1.975,0.336



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,24.569,1.728


Embedded Clause

In [40]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [41]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed')


=== ADJUNCT ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0952
Min. group size: 12
Log-Likelihood: -124.8343
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.444,0.194,38.322,0.000,7.063,7.825
1,adjunct_type[T.back],0.077,0.049,1.568,0.117,-0.019,0.172
2,adjunct_type[T.front],0.539,0.049,11.038,0.000,0.443,0.634
3,wh_numeric,0.155,0.035,4.504,0.000,0.088,0.223
4,wh_numeric:adjunct_type[T.back],-0.076,0.049,-1.554,0.120,-0.171,0.020
5,wh_numeric:adjunct_type[T.front],0.076,0.049,1.564,0.118,-0.019,0.172
6,gap_numeric,-0.037,0.035,-1.080,0.280,-0.105,0.030
7,gap_numeric:adjunct_type[T.back],0.080,0.049,1.646,0.100,-0.015,0.176
8,gap_numeric:adjunct_type[T.front],0.064,0.049,1.314,0.189,-0.032,0.160
9,wh_numeric:gap_numeric,-0.067,0.035,-1.941,0.052,-0.135,0.001



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.731,0.811


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands: subject condition')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands: object condition')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause subject condition')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause object condition')